## Figure objects merge solution
## Cross correlation

In [1]:
import matplotlib
matplotlib.use('Agg')
#https://stackoverflow.com/questions/37604289/tkinter-tclerror-no-display-name-and-no-display-environment-variable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
#https://stackoverflow.com/questions/27476642/matplotlib-get-rid-of-max-open-warning-output

from biosppy.signals import ecg
import os
import scipy as sp
import datetime

In [7]:
os.mkdir('/dscrhome/yq59/data/ECG Strips/70')

In [2]:
ecg_data_file = '/dscrhome/yq59/data/iBeat/eplab/Subject70_data.hdf'
#ecg_data_file = 'D:/data/iBeat/eplab/Subject44_data.hdf'
ecg_data = pd.read_hdf(ecg_data_file,'eplab')

In [3]:
print('read file')

read file


In [4]:
#ecg_data = ecg_data[(ecg_data['utc_adjusted'] < 1708138200)& (ecg_data['utc_adjusted'] > 1708145800)]

In [6]:
print('cut data')

cut data


In [5]:
SR = 976.942

out = ecg.ecg(signal=ecg_data.II, sampling_rate=SR,show = False)
ecg_sig_II = out['filtered']
rpeaks2 = out['rpeaks']

ecg_sig_V6 = ecg.ecg(signal=ecg_data.V6, sampling_rate=SR,show = False)['filtered']

C:\Users\admin\Anaconda3\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
C:\Users\admin\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
C:\Users\admin\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index

In [8]:
print('Baseline correction and rpeak done')

Baseline correction and rpeak done


In [8]:
rrinterval = []
for i in range(len(rpeaks2)-1):
    rrinterval.append((rpeaks2[i+1]-rpeaks2[i])/SR)
#print(len(rrinterval))

In [9]:
thold_num = 5
index = []
for i in range(len(rrinterval)-thold_num):
    if rrinterval[i] < 0.6 and rrinterval[i+1] < 0.6 and rrinterval[i+2] < 0.6 and rrinterval[i+3] < 0.6 and rrinterval[i+4] < 0.6:
        index.append(i)

In [10]:
import more_itertools as mit
grouped_index = [list(group) for group in mit.consecutive_groups(index)]
#grouped_index[0:5]
print(len(grouped_index) , 'tachycardia events extracted')

21 tachycardia events extracted


In [11]:
begin_index = []
end_index = []
for i in range(len(grouped_index)):
    begin_index.append(grouped_index[i][0])
    end_index.append(grouped_index[i][-1] + thold_num)

In [12]:
print('indices ready')

indices ready


In [31]:
ecg_sig = {'II':ecg_sig_II,'V6':ecg_sig_V6}

major_ticks = np.arange(0, 10, 1)
minor_ticks = np.arange(0, 10, 0.2)

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

for sig in ecg_sig:
    
    for i in range(len(begin_index)):
    
    # To plot same line with different colors. https://matplotlib.org/1.4.1/examples/pylab_examples/multicolored_line.html
    # Use locals() function to set variable names iteratively, such as: tachy_begin1, tachy_begin2, ...
    # https://docs.python.org/3/library/functions.html#locals
    # Note that the ith RRinterval is calculated by (i+1)th rpeak minus the ith rpeak.
    # Tachycardia initiates at 5sec location.
        
        locals()['tachy_begin'+str(i)] = ecg_sig[sig][(rpeaks2[begin_index[i]] - round(5*SR)):(rpeaks2[begin_index[i]] + round(5*SR))]

        t = np.linspace(0,10,num = round(10*SR) + 1)
        cmap = ListedColormap(['b', 'r', 'b'])

        locals()['cutpoint_b'+str(i)] = (rpeaks2[end_index[i]] - rpeaks2[begin_index[i]])/SR + 5
        locals()['norm_b'+str(i)] = BoundaryNorm([0, 5, locals()['cutpoint_b'+str(i)], 10], cmap.N)
        locals()['points_b'+str(i)] = np.array([t, locals()['tachy_begin'+str(i)]]).T.reshape(-1, 1, 2)
        locals()['segments_b'+str(i)] = np.concatenate([locals()['points_b'+str(i)][:-1], locals()['points_b'+str(i)][1:]], axis=1)

        locals()['lc_b'+str(i)] = LineCollection(locals()['segments_b'+str(i)], cmap=cmap, norm=locals()['norm_b'+str(i)])
        locals()['lc_b'+str(i)].set_array(t)

        locals()['fig_b'+str(i)] = plt.figure(figsize = (20,2))
        locals()['fig_b'+str(i)].gca().add_collection(locals()['lc_b'+str(i)])         # GetCurrentAxis

        locals()['ax_b'+str(i)] = locals()['fig_b'+str(i)].add_subplot(1, 1, 1)

        locals()['ax_b'+str(i)].set_xticks(major_ticks)
        locals()['ax_b'+str(i)].set_xticks(minor_ticks, minor=True)
        locals()['ax_b'+str(i)].grid(which='minor', alpha=0.5, linestyle='--')
        locals()['ax_b'+str(i)].grid(which='major', alpha=1, linestyle='-')
        
        plt.ylim(min(locals()['tachy_begin'+str(i)])-0.1, max(locals()['tachy_begin'+str(i)])+0.1)    
        plt.xlim(0,10)
        plt.xlabel('{}_{}_{}_begin@{}'.format(70, i+1, sig,datetime.datetime.fromtimestamp(ecg_data.iloc[rpeaks2[begin_index[i]]]['utc_adjusted']).strftime("%Y-%m-%d %H:%M:%S")) + '    Time/sec')
        plt.ylabel('Adjusted_ECG_amplitude_{}'.format(sig))
        
          #Save subplots to *.png
        locals()['bbox_b'+str(i)] = locals()['ax_b'+str(i)].get_tightbbox(locals()['fig_b'+str(i)].canvas.get_renderer())
        locals()['fig_b'+str(i)].savefig("/dscrhome/yq59/data/ECG Strips/70/70_{}_{}_begin.png".format(i+1, sig), 
                   bbox_inches=locals()['bbox_b'+str(i)].transformed(locals()['fig_b'+str(i)].dpi_scale_trans.inverted()))
        
        plt.close()


In [15]:
print('begin figure saved')

begin figure saved


In [32]:
for sig in ecg_sig:
    
    for i in range(len(begin_index)):

        locals()['tachy_end'+str(i)] = ecg_sig[sig][(rpeaks2[end_index[i]] - round(5*SR)):(rpeaks2[end_index[i]] + round(5*SR))]

        t = np.linspace(0,10,num = round(10*SR) + 1)

        locals()['cutpoint_e'+str(i)] = 5 - (rpeaks2[end_index[i]] - rpeaks2[begin_index[i]])/SR 

        cmap = ListedColormap(['b', 'r', 'b'])
        locals()['norm_e'+str(i)] = BoundaryNorm([0, locals()['cutpoint_e'+str(i)], 5, 10], cmap.N)

        locals()['points_e'+str(i)] = np.array([t, locals()['tachy_end'+str(i)]]).T.reshape(-1, 1, 2)
        locals()['segments_e'+str(i)] = np.concatenate([locals()['points_e'+str(i)][:-1], locals()['points_e'+str(i)][1:]], axis=1)

        locals()['lc_e'+str(i)] = LineCollection(locals()['segments_e'+str(i)], cmap=cmap, norm=locals()['norm_e'+str(i)])
        locals()['lc_e'+str(i)].set_array(t)

        locals()['fig_e'+str(i)] = plt.figure(figsize = (20,2))
        locals()['ax_e'+str(i)] = locals()['fig_e'+str(i)].add_subplot(1, 1, 1)

        locals()['fig_e'+str(i)].gca().add_collection(locals()['lc_e'+str(i)])

        locals()['ax_e'+str(i)].set_xticks(major_ticks)
        locals()['ax_e'+str(i)].set_xticks(minor_ticks, minor=True)
        locals()['ax_e'+str(i)].grid(which='minor', alpha=0.5, linestyle='--')
        locals()['ax_e'+str(i)].grid(which='major', alpha=1, linestyle='-')

        plt.ylim(min(locals()['tachy_end'+str(i)])-0.1, max(locals()['tachy_end'+str(i)])+0.1)     
        plt.xlim(0,10)
        plt.xlabel('{}_{}_{}_end@{}'.format(70, i+1, sig, datetime.datetime.fromtimestamp(ecg_data.iloc[rpeaks2[end_index[i]]]['utc_adjusted']).strftime("%Y-%m-%d %H:%M:%S")) + '    Time/sec')
        plt.ylabel('Adjusted_ECG_amplitude_{}'.format(sig))
        
        locals()['bbox_e'+str(i)] = locals()['ax_e'+str(i)].get_tightbbox(locals()['fig_e'+str(i)].canvas.get_renderer())
        locals()['fig_e'+str(i)].savefig("/dscrhome/yq59/data/ECG Strips/70/70_{}_{}_end.png".format(i+1, sig),
                                         bbox_inches=locals()['bbox_e'+str(i)].transformed(locals()['fig_e'+str(i)].dpi_scale_trans.inverted()))
       
        plt.close()  
        #Close figure in the loop.


In [17]:
print('end figure saved')

end figure saved


In [18]:
# fig, ax = plt.subplots(len(begin_index), 1, sharex=False, figsize=(20, 10))
for i in range(len(begin_index)):
    locals()['rr'+str(i)] = []
    locals()['rr'+str(i)].append(rrinterval[begin_index[i]:end_index[i]])
    locals()['bpm'+str(i)] = []
    
    for k in locals()['rr'+str(i)][0]:
        locals()['bpm'+str(i)].append(60/k)
        
    locals()['time'+str(i)] = (rpeaks2[end_index[i]] - rpeaks2[begin_index[i]])/SR
    
    locals()['t'+str(i)] = np.linspace(0,locals()['time'+str(i)], num = len(locals()['rr'+str(i)][0]))
    
    locals()['fig'+str(i)] = plt.figure(figsize = (20,2))
    locals()['ax'+str(i)] = locals()['fig'+str(i)].add_subplot(1, 1, 1)
    
    locals()['ax'+str(i)].plot(locals()['t'+str(i)] , locals()['bpm'+str(i)])
    
#     ax[i].plot(locals()['t'+str(i)] , locals()['rr'+str(i)][0])
    plt.xlabel('{}_{}_BPM'.format(70,i+1) + '    Time/sec')
    plt.ylabel('beats per minute')
    
    locals()['bbox'+str(i)] = locals()['ax'+str(i)].get_tightbbox(locals()['fig'+str(i)].canvas.get_renderer())
    locals()['fig'+str(i)].savefig("/dscrhome/yq59/data/ECG Strips/70/70_{}_RRinterval.png".format(i+1),
              bbox_inches=locals()['bbox'+str(i)].transformed(locals()['fig'+str(i)].dpi_scale_trans.inverted()))
    
    plt.close()  #Close figure in the loop.

In [19]:
print('RRintv figure saved')

RRintv figure saved


In [20]:

min_index = []
# minimum RR interval after tachycardia begins, begin_index[i] + min_index[i] + 1

for sig in ecg_sig:

    for i in range(len(begin_index)):

        min_index.append(np.argmin(locals()['rr'+str(i)])) 
        t = np.linspace(0,10,num = round(10*SR) + 1)

        locals()['tachy_mid'+str(i)] = ecg_sig[sig][(rpeaks2[begin_index[i] + min_index[i] + 1] - round(5*SR)):(rpeaks2[begin_index[i] + min_index[i] + 1] + round(5*SR))]

        locals()['left_cutpoint'+str(i)] = 5 - (rpeaks2[begin_index[i] + min_index[i] + 1] - rpeaks2[begin_index[i]])/SR 
        locals()['right_cutpoint'+str(i)] = (rpeaks2[end_index[i]] - rpeaks2[begin_index[i] + min_index[i] + 1])/SR + 5

        #ax[i].plot(t, locals()['tachy_mid'+str(i)])

        cmap = ListedColormap(['b', 'r', 'b'])
        locals()['norm'+str(i)] = BoundaryNorm([0, locals()['left_cutpoint'+str(i)], locals()['right_cutpoint'+str(i)], 10], cmap.N)

        locals()['points'+str(i)] = np.array([t, locals()['tachy_mid'+str(i)]]).T.reshape(-1, 1, 2)
        locals()['segments'+str(i)] = np.concatenate([locals()['points'+str(i)][:-1], locals()['points'+str(i)][1:]], axis=1)

        locals()['lc'+str(i)] = LineCollection(locals()['segments'+str(i)], cmap=cmap, norm=locals()['norm'+str(i)])
        locals()['lc'+str(i)].set_array(t)

        locals()['fig'+str(i)] = plt.figure(figsize = (20,2))
        locals()['ax'+str(i)] = locals()['fig'+str(i)].add_subplot(1, 1, 1)

        locals()['fig'+str(i)].gca().add_collection(locals()['lc'+str(i)])

        locals()['ax'+str(i)].set_xticks(major_ticks)
        locals()['ax'+str(i)].set_xticks(minor_ticks, minor=True)
        locals()['ax'+str(i)].grid(which='minor', alpha=0.5, linestyle='--')
        locals()['ax'+str(i)].grid(which='major', alpha=1, linestyle='-')

        plt.ylim(min(locals()['tachy_mid'+str(i)])-0.1, max(locals()['tachy_mid'+str(i)])+0.1)    
        plt.xlim(0,10)
        plt.ylabel('Adjusted_ECG_amplitude_{}'.format(sig))
        plt.xlabel('70_{}_{}_fastestHR@{}'.format(i+1, sig, 
                                               datetime.datetime.fromtimestamp(ecg_data.iloc[rpeaks2[begin_index[i] + min_index[i] + 1]]['utc_adjusted']).strftime("%Y-%m-%d %H:%M:%S")) + '    Time/sec')

        #Save subplots to png, 
        locals()['bbox'+str(i)] = locals()['ax'+str(i)].get_tightbbox(locals()['fig'+str(i)].canvas.get_renderer())
        locals()['fig'+str(i)].savefig("/dscrhome/yq59/data/ECG Strips/70/70_{}_{}_fastestHR.png".format(i+1, sig),
                  bbox_inches=locals()['bbox'+str(i)].transformed(locals()['fig'+str(i)].dpi_scale_trans.inverted()))
        
        plt.close()  #Close figure in the loop.

In [21]:
print('fastest HR figure saved')

fastest HR figure saved
